# Problem description

We will look at how to extract information from an interaction between Customer and an Agent. This might be useful in build a call center analysis dashboard kind of applicaiton.

In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

/data/conda_env/yash/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Loading and exploring the dataset

In [2]:
data = load_dataset("jonathansuru/customer_service_information_extraction", cache_dir = "/data/datasets")

In [3]:
data

DatasetDict({
    train: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 190
    })
})

In [4]:
print(data['train']['prompt'][0])

Please extract the customer specifications from the conversation below:

###

Agent: Hello, thank you for calling [Company Name]. How may I help you today?
Customer: I'm calling to complain about a product that I purchased.
Agent: I understand. Can you please tell me what product you're referring to?
Customer: I purchased a [Product Name] from your website on August 10th. It arrived on August 15th, but it was damaged.
Agent: I'm sorry to hear that. I'll be happy to help you with this. Can you send me a picture of the damaged product?
Customer: Sure.
...
Agent: I've received the picture of the damaged product. I'm going to issue you a refund for the product. I'm also going to send you a replacement product.
Customer: Thank you for your help.
The extract is as follows:


In [5]:
print(data['train']['completion'][0])

 "product name": "product name",
  "issue": "damaged product"
END


## Zero Shot Setting (with base model)

In [6]:
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-2-7b-hf",
    cache_dir="/data/base_models/",
    device_map='cuda:2',
)

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf", cache_dir="/data/base_models/")

Loading checkpoint shards: 100%|████████████████████████████████████████████████████████████████████████████████| 2/2 [00:05<00:00,  2.66s/it]


In [7]:
def get_llama2_reponse(prompt, max_new_tokens=50):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=max_new_tokens, temperature= 0.00001)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

In [8]:
prompt = data['train']['prompt'][0]
print(get_llama2_reponse(prompt))

Please extract the customer specifications from the conversation below:

###

Agent: Hello, thank you for calling [Company Name]. How may I help you today?
Customer: I'm calling to complain about a product that I purchased.
Agent: I understand. Can you please tell me what product you're referring to?
Customer: I purchased a [Product Name] from your website on August 10th. It arrived on August 15th, but it was damaged.
Agent: I'm sorry to hear that. I'll be happy to help you with this. Can you send me a picture of the damaged product?
Customer: Sure.
...
Agent: I've received the picture of the damaged product. I'm going to issue you a refund for the product. I'm also going to send you a replacement product.
Customer: Thank you for your help.
The extract is as follows:

###

Agent: Hello, thank you for calling [Company Name]. How may I help you today?
Customer: I'm calling to complain about a product that I purchased.
Agent: I understand. Can you


## Zero Shot Setting (with chat finetuned model)

In [9]:
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-2-7b-chat-hf",
    cache_dir="/data/base_models/",
    device_map='cuda:2',
)

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf", cache_dir="/data/base_models/")

Loading checkpoint shards: 100%|████████████████████████████████████████████████████████████████████████████████| 2/2 [00:05<00:00,  2.52s/it]


In [10]:
prompt = data['train']['prompt'][0]
print(get_llama2_reponse(prompt))

Please extract the customer specifications from the conversation below:

###

Agent: Hello, thank you for calling [Company Name]. How may I help you today?
Customer: I'm calling to complain about a product that I purchased.
Agent: I understand. Can you please tell me what product you're referring to?
Customer: I purchased a [Product Name] from your website on August 10th. It arrived on August 15th, but it was damaged.
Agent: I'm sorry to hear that. I'll be happy to help you with this. Can you send me a picture of the damaged product?
Customer: Sure.
...
Agent: I've received the picture of the damaged product. I'm going to issue you a refund for the product. I'm also going to send you a replacement product.
Customer: Thank you for your help.
The extract is as follows:

* Product: [Product Name]
* Date of purchase: August 10th
* Date of delivery: August 15th
* Damage: Damaged
* Action taken by agent: Issued a refund


The chat finetuned model does seem to give good results without any kind of training. We can do a bit of prompt engineering and improve the reuslts. Lets assume we tried all techniques but we still dont get desired output (that exact attributes and values that we want in desired format). We will go for fine-tuning Llama-2 model by ourselves using the dataset.